In [1]:
# Cell 0: RAG Initialization (Run First)
# -------------------------------------
# Sets up Weaviate collection using text2vec-transformers (external inference container).
# Ensure docker-compose is up with services: weaviate + t2v-transformers.
#   ENABLE_MODULES=text2vec-transformers,generative-openai
#   DEFAULT_VECTORIZER_MODULE=text2vec-transformers
#   TRANSFORMERS_INFERENCE_API=http://t2v-transformers:8080
#   (If you previously used generative-ollama, switch to generative-openai here.)
#
# OpenAI setup: keep these environment variables available (e.g., in a .env file):
#   OPENAI_API_KEY=sk-...
#   # Optional if using a proxy or Azure-compatible endpoint:
#   OPENAI_BASE_URL=https://api.openai.com/v1
#   OPENAI_MODEL=gpt-4o-mini
#
# Timeout tuning: Some weaviate client versions expose Timeout; if not, we fall back.

# Increase default timeouts to reduce GRPC RST_STREAM errors during heavy queries
CUSTOM_INIT_TIMEOUT = 120
CUSTOM_QUERY_TIMEOUT = 900   # 15 minutes
CUSTOM_INSERT_TIMEOUT = 300  # 5 minutes

from email import header
from dotenv import load_dotenv
import os
import weaviate
from weaviate.classes.config import Configure, Property, DataType, Tokenization
from weaviate.config import AdditionalConfig, Timeout
openai_api_key = os.getenv("OPENAI_API_KEY")
headers = {
    "X-Openai-Api-Key": openai_api_key
}
# Attempt optional Timeout (newer weaviate client). If missing, continue with defaults.
try:
    TIMEOUTS = Timeout(init=CUSTOM_INIT_TIMEOUT, query=CUSTOM_QUERY_TIMEOUT, insert=CUSTOM_INSERT_TIMEOUT)
    client = weaviate.connect_to_local(
        additional_config=AdditionalConfig(timeout=TIMEOUTS),
        headers=headers
        )
    print("Custom timeouts applied:", TIMEOUTS)
except ImportError:
    client = weaviate.connect_to_local()
    TIMEOUTS = None
    print("Timeout not available in this weaviate version; using default client timeouts.")
    print("Tip: pip install --upgrade weaviate-client to enable configurable timeouts.")
except TypeError:
    # Signature mismatch (older version). Reconnect without custom timeouts.
    client = weaviate.connect_to_local()
    TIMEOUTS = None
    print("Timeout signature unsupported; using default timeouts.")
    print("Tip: pip install --upgrade weaviate-client to enable configurable timeouts.")

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL_NAME = os.getenv("OPENAI_MODEL", "gpt-4o-mini")  # for reference in prompts

load_dotenv()

# Diagnostics
try:
    meta = client.get_meta()
    print("Server modules detected:", list(meta.get("modules", {}).keys()))
except Exception as e:
    print("Meta fetch failed:", e)

# Recreate collection for a clean slate
try:
    client.collections.delete("Dataset")
except Exception:
    pass

# Create collection with OpenAI as the generative provider (no Flex args)
openai_model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

gen_cfg = Configure.Generative.openai(
    model=openai_model
)

client.collections.create(
    "Dataset",
    properties=[
        Property(name="text", data_type=DataType.TEXT, tokenization=Tokenization.LOWERCASE),
        Property(name="file_path", data_type=DataType.TEXT)
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_transformers(
            name="text_vector",
            source_properties=["text"],
            pooling_strategy="masked_mean",
        )
    ],
    generative_config=gen_cfg
)
assert client.collections.exists("Dataset")
print("Collection 'Dataset' ready (text2vec-transformers + generative-openai).")

Custom timeouts applied: query=900 insert=300 init=120
Server modules detected: ['generative-anthropic', 'generative-anyscale', 'generative-aws', 'generative-cohere', 'generative-databricks', 'generative-friendliai', 'generative-google', 'generative-mistral', 'generative-nvidia', 'generative-octoai', 'generative-ollama', 'generative-openai', 'generative-xai', 'multi2multivec-jinaai', 'multi2vec-cohere', 'multi2vec-google', 'multi2vec-jinaai', 'multi2vec-nvidia', 'multi2vec-voyageai', 'reranker-cohere', 'reranker-jinaai', 'reranker-nvidia', 'reranker-voyageai', 'text2multivec-jinaai', 'text2vec-aws', 'text2vec-cohere', 'text2vec-databricks', 'text2vec-google', 'text2vec-huggingface', 'text2vec-jinaai', 'text2vec-mistral', 'text2vec-nvidia', 'text2vec-octoai', 'text2vec-openai', 'text2vec-transformers', 'text2vec-voyageai', 'text2vec-weaviate']


c:\Users\Francisco Azeredo\.conda\envs\tese\Lib\site-packages\weaviate\warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(


Collection 'Dataset' ready (text2vec-transformers + generative-openai).


In [2]:
import torch
import os, json, random, time, gc
from pathlib import Path
import psutil
from tqdm.auto import tqdm
from docx import Document
# ---------------- User Config ----------------
SHUFFLE_DOCS = True
MAX_DOCS = None  # set int to limit docs
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
DATASET_DIR = r"C:\Users\Francisco Azeredo\Downloads\gerador_documentos_gpt_azure (1)\gerador_documentos_gpt_azure\documentos_gerados"
WORKING_DIR = r"C:\Users\Francisco Azeredo\Downloads\gerador_documentos_gpt_azure (1)\gerador_documentos_gpt_azure"
LLM_MODEL_NAME = "qwen2.5:latest"
LOG_LEVEL = "CRITICAL"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

try:
    PROCESS = psutil.Process()
except Exception:
    PROCESS = None

# ---------------- Helpers ----------------

def memory_mb():
    if PROCESS is None: return None
    return PROCESS.memory_info().rss / (1024 * 1024)

def read_text_from_file(path: Path) -> str:
    suffix = path.suffix.lower()
    try:
        if suffix in {".txt", ".md"}:
            return path.read_text(encoding="utf-8", errors="ignore")
        if suffix == ".json":
            data = json.loads(path.read_text(encoding="utf-8", errors="ignore"))
            for k in ("text","content","body","article"):
                if isinstance(data, dict) and k in data and isinstance(data[k], str):
                    return data[k]
            return json.dumps(data)
        if suffix in {".jsonl", ".ndjson"}:
            lines = []
            with path.open("r", encoding="utf-8", errors="ignore") as f:
                for line in f:
                    line=line.strip()
                    if not line: continue
                    try:
                        obj=json.loads(line)
                        if isinstance(obj, dict):
                            for k in ("text","content","body","article"):
                                if k in obj and isinstance(obj[k], str):
                                    lines.append(obj[k]); break
                            else:
                                lines.append(json.dumps(obj))
                        else:
                            lines.append(str(obj))
                    except Exception:
                        lines.append(line)
            return "\n".join(lines)
        if suffix == ".docx":
            # Read .docx files using python-docx
            doc = Document(path)
            text = []
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    text.append(paragraph.text.strip())
            return "\n".join(text)
    except Exception as e:
        return f"ERROR_READING_FILE {path.name}: {e}"
    return ""

def load_documents(root_dir: str):
    exts = (".txt", ".md", ".json", ".jsonl", ".ndjson", ".docx")
    paths = [p for p in Path(root_dir).rglob("*") if p.suffix.lower() in exts and p.is_file()]
    if SHUFFLE_DOCS: random.shuffle(paths)
    docs = []
    for p in paths:
        if MAX_DOCS and len(docs) >= MAX_DOCS: break
        text = read_text_from_file(p).strip()
        if not text: continue
        docs.append({"id": f"doc_{len(docs)}", "text": text, "source_path": str(p)})
    return docs

# ---------------- Indexing ----------------
async def index_documents(rag):
    print("Loading documents...")
    t0 = time.perf_counter(); docs = load_documents(DATASET_DIR)
    print(f"Loaded {len(docs)} docs in {time.perf_counter()-t0:.2f}s")
    if not docs:
        print("No documents found; adjust DATASET_DIR."); return
    start_mem = memory_mb()
    if start_mem is not None: print(f"Start RSS: {start_mem:.2f} MB")
    texts = [d['text'] for d in docs]
    metas = [{"id": d['id'], "source": d['source_path']} for d in docs]
    print("Indexing (batched)...")
    t1 = time.perf_counter()
    failed = 0
    with rag.batch.dynamic() as batch:
        for text, metadata in tqdm(zip(texts, metas), desc="Indexing", total=len(texts)):
            try:
                batch.add_object(
                    properties={
                        "text": text,
                        "file_path": metadata.get("source")
                    }
                )
            except Exception as e:
                failed += 1
                if failed < 5:
                    print(f"Failed {metadata.get('id')}: {e}")
    dur = time.perf_counter()-t1
    print(f"Inserted {len(texts)-failed} / {len(texts)} docs in {dur:.2f}s ({((len(texts)-failed)/dur) if dur>0 else 0:.2f} docs/s)")
    if failed:
        print(f"Total failed: {failed}")
    gc.collect(); end_mem = memory_mb()
    if end_mem is not None: print(f"End RSS: {end_mem:.2f} MB (Δ {end_mem - start_mem:.2f} MB)")

rag = client.collections.get("Dataset")
await index_documents(rag)
print("Indexing complete. Proceed to Cell 2 for querying & evaluation.")
client.close()

Device: cuda
Loading documents...
Loaded 1000 docs in 5.81s
Start RSS: 686.73 MB
Indexing (batched)...
Loaded 1000 docs in 5.81s
Start RSS: 686.73 MB
Indexing (batched)...


Indexing:   0%|          | 0/1000 [00:00<?, ?it/s]

Inserted 1000 / 1000 docs in 10.73s (93.18 docs/s)
End RSS: 585.62 MB (Δ -101.11 MB)
Indexing complete. Proceed to Cell 2 for querying & evaluation.


In [ ]:
# Cell 1: Warm up Ollama (Optional)
# ----------------------------------
# Ensures the Ollama model is loaded before Weaviate calls it.

# You can rerun this cell anytime after starting Ollama or changing models.

import time
import requests

# Configure warm-up behavior
AUTO_WARMUP_OLLAMA = False
WARMUP_TIMEOUT = 600          # seconds to wait for model to be ready
WARMUP_POLL_INTERVAL = 5      # seconds between checks
WARMUP_API_URL = "http://localhost:11434/api/generate"  # Ollama generate endpoint

def warm_up_ollama(model: str | None = None, timeout_s: int = WARMUP_TIMEOUT, poll_interval: int = WARMUP_POLL_INTERVAL, prompt: str = "ping", api_url: str = WARMUP_API_URL) -> bool:
    """Ping Ollama generate API until the model is ready or timeout expires.
    Returns True when ready, False if timed out."""
    mdl = model or globals().get("LLM_MODEL_NAME", "qwen2.5:latest")
    print(f"🔧 Warming up Ollama model: {mdl}")
    start = time.perf_counter()
    last_err = None
    while (time.perf_counter() - start) < timeout_s:
        try:
            resp = requests.post(api_url, json={"model": mdl, "prompt": prompt, "stream": False}, timeout=3000)
            if resp.status_code == 200:
                try:
                    data = resp.json()
                except Exception:
                    data = {}
                text = data.get("response") or data.get("message") or ""
                print("✅ Ollama ready.")
                return True
            else:
                last_err = f"HTTP {resp.status_code}: {resp.text[:200]}"
        except Exception as e:
            last_err = str(e)
        print(f"…waiting for Ollama to load model (poll {poll_interval}s). Last error: {last_err}")
        time.sleep(poll_interval)
    print(f"⏱️ Warm-up timed out after {timeout_s}s. Last error: {last_err}")
    return False

# Auto warm-up when this cell runs
if AUTO_WARMUP_OLLAMA:
    warm_up_ollama()

🔧 Warming up Ollama model: qwen2.5:latest
✅ Ollama ready.
✅ Ollama ready.


In [ ]:
# Cell 2: Query & QA Evaluation - Best Metrics Only
# ----------------------------------------------
# Run AFTER Cell 1. Uses the global `rag` object and indexed data.
# Optimized to focus on the most meaningful metrics for RAG evaluation.

from pickle import FALSE
import os, csv, time, json, random, re, statistics, asyncio, math
from pathlib import Path
from nltk.metrics import edit_distance
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize, sent_tokenize
from regex import F
from rouge import Rouge
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm.auto import tqdm

# -------- Configuration - Using Relative Paths --------
# Updated to use relative paths after reorganization
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

QA_JSON_PATH = str(PROJECT_ROOT / "datasets" / "qa_dataset300.json")
OUTPUT_CSV_PATH = str(PROJECT_ROOT / "results_csv")  # set to None to skip saving
TOP_K = 3            # lower to reduce vector fetch time
MAX_Q = None         # limit question count
RANDOM_SEED = 42
USE_BERT_SIM = True  # semantic metrics cost
PER_QUERY_DEADLINE = 60000.0  # seconds, must be < client.query timeout
MAX_RETRIES = 3
RETRY_BACKOFF = 10.0  # seconds added each retry
LANGUAGE = "pt-PT"
PROMPT_PREFIX = "Responda de forma breve e objetiva em português (pt-PT): "
COT_PROMPT = False  # chain-of-thought prompting (slower, may improve complex Qs)
AUTO_WARMUP_OLLAMA = globals().get('AUTO_WARMUP_OLLAMA', True)
random.seed(RANDOM_SEED)

# Verify paths
print(f"✓ Paths configured:")
print(f"  Q&A Dataset: {QA_JSON_PATH}")
print(f"  Output Folder: {OUTPUT_CSV_PATH}")
print(f"  Dataset exists: {Path(QA_JSON_PATH).exists()}")
print(f"  Output folder exists: {Path(OUTPUT_CSV_PATH).exists()}")

TOKEN_SPLIT_RE = re.compile(r"\W+", re.UNICODE)
_ROUGE = None
_BERT_MODEL = None
_SMOOTH = SmoothingFunction().method1

def _lazy_rouge():
    global _ROUGE
    if _ROUGE is None:
        _ROUGE = Rouge()
    return _ROUGE

def _lazy_bert():
    global _BERT_MODEL
    if _BERT_MODEL is None:
        _BERT_MODEL = SentenceTransformer('all-MiniLM-L6-v2')
    return _BERT_MODEL

def tokenize_pt(s: str):
    return [t.lower() for t in TOKEN_SPLIT_RE.split(s) if t.strip()]

def token_recall(answer: str, gold: str) -> float:
    at = set(tokenize_pt(answer))
    gt = set(tokenize_pt(gold))
    if not gt:
        return 0.0
    return len(at & gt) / len(gt)

def compute_rouge1(hyp: str, ref: str):
    if not hyp.strip() or not ref.strip():
        return {"rouge-1": {"f": 0.0, "p": 0.0, "r": 0.0}}
    try:
        return _lazy_rouge().get_scores(hyp, ref)[0]
    except:
        return {"rouge-1": {"f": 0.0, "p": 0.0, "r": 0.0}}

def compute_bleu(hyp: str, ref: str) -> float:
    h_toks = tokenize_pt(hyp)
    r_toks = tokenize_pt(ref)
    if not h_toks or not r_toks:
        return 0.0
    try:
        return sentence_bleu([r_toks], h_toks, smoothing_function=_SMOOTH)
    except:
        return 0.0

def compute_bert_similarity(hyp: str, ref: str) -> float:
    if not USE_BERT_SIM or not hyp.strip() or not ref.strip():
        return 0.0
    model = _lazy_bert()
    emb = model.encode([hyp, ref])
    return float(np.dot(emb[0], emb[1]) / (np.linalg.norm(emb[0]) * np.linalg.norm(emb[1])))

def best_sentence_overlap(answer: str, gold: str) -> float:
    sents_a = sent_tokenize(answer, language="portuguese")
    sents_g = sent_tokenize(gold, language="portuguese")
    best = 0.0
    for sa in sents_a:
        for sg in sents_g:
            tr = token_recall(sa, sg)
            if tr > best:
                best = tr
    return best

# Load QA
qa_pairs = []
if os.path.exists(QA_JSON_PATH):
    with open(QA_JSON_PATH, encoding="utf-8") as f:
        qa_data = json.load(f)
        for item in qa_data:
            if "pergunta" in item and "resposta" in item:
                qa_pairs.append({
                    "question": item["pergunta"].strip(),
                    "gold_answer": item["resposta"].strip(),
                    "context": item.get("contexto","").strip(),
                    "file": item.get("arquivo","").strip()
                })
else:
    print("QA JSON not found. Check QA_JSON_PATH.")

if MAX_Q:
    random.shuffle(qa_pairs)
    qa_pairs = qa_pairs[:MAX_Q]

print(f"Loaded {len(qa_pairs)} Q&A pairs.")

# Optional warmup
if AUTO_WARMUP_OLLAMA:
    warm_up_ollama()

rows = []
latencies = []
correct_retrievals = 0
cot_phrase = " Vamos pensar passo a passo." if COT_PROMPT else ""

for idx, pair in enumerate(tqdm(qa_pairs, desc="Evaluating (Best Metrics)")):
    q = pair["question"]
    gold_answer = pair["gold_answer"]
    gold_context = pair["context"]
    gold_file = pair["file"]
    
    start_t = time.perf_counter()
    try:
        response_obj = rag.query(q + cot_phrase, param={"limit": TOP_K})
        
        if not response_obj or not hasattr(response_obj, "answer"):
            print(f"Q#{idx+1}: No valid response. Skipping.")
            continue
        
        answer_text = response_obj.answer.strip() if response_obj.answer else ""
        retrieved_context = " ".join([ctx.get("content", "") for ctx in response_obj.context]) if hasattr(response_obj, "context") else ""
        
        # Retrieval metrics
        retrieved_files = [ctx.get("file_path", "") for ctx in response_obj.context] if hasattr(response_obj, "context") else []
        rank = retrieved_files.index(gold_file) if gold_file in retrieved_files else -1
        if rank >= 0:
            correct_retrievals += 1
        
        # Answer quality (best metrics)
        exact = 1.0 if answer_text.lower() == gold_answer.lower() else 0.0
        substring = 1.0 if gold_answer.lower() in answer_text.lower() else 0.0
        tok_recall = token_recall(answer_text, gold_answer)
        rouge1_scores = compute_rouge1(answer_text, gold_answer)
        rouge1_f = rouge1_scores["rouge-1"]["f"]
        bert_cos = compute_bert_similarity(answer_text, gold_answer) if USE_BERT_SIM else 0.0
        
        # Context quality
        context_token_recall = token_recall(retrieved_context, gold_context) if gold_context else 0.0
        context_rouge1 = compute_rouge1(retrieved_context, gold_context) if gold_context else {"rouge-1": {"f": 0.0}}
        context_bert = compute_bert_similarity(retrieved_context, gold_context) if USE_BERT_SIM and gold_context else 0.0
        
        lat = time.perf_counter() - start_t
        latencies.append(lat)
        
        rows.append({
            "question": q,
            "gold_answer": gold_answer,
            "generated_answer": answer_text,
            "gold_file": gold_file,
            "retrieved_files": "|".join(retrieved_files),
            "retrieval_rank": rank,
            "exact": exact,
            "substring": substring,
            "token_recall": tok_recall,
            "rouge1_f": rouge1_f,
            "bert_cos": bert_cos,
            "context_token_recall": context_token_recall,
            "context_rouge1_f": context_rouge1["rouge-1"]["f"],
            "context_bert_cos": context_bert,
            "latency_s": lat
        })
        
    except Exception as e:
        print(f"Q#{idx+1} error: {e}")
        continue

# Results summary
if rows:
    def _avg(key):
        vals = [r[key] for r in rows if key in r and isinstance(r[key], (int,float))]
        return sum(vals)/len(vals) if vals else 0.0
    
    # Report best metrics only
    print(f"\n{'='*60}")
    print(f"RAG EVALUATION RESULTS - BEST METRICS ONLY")
    print(f"{'='*60}")
    
    print(f"\n🎯 RETRIEVAL PERFORMANCE:")
    print(f"  Document Retrieval Accuracy: {correct_retrievals}/{len(qa_pairs)} = {correct_retrievals/len(qa_pairs):.2%}")
    print(f"  Average Retrieval Rank: {_avg('retrieval_rank'):.1f}")
    
    print(f"\n📝 ANSWER QUALITY:")
    print(f"  Exact Match: {_avg('exact'):.2%}")
    print(f"  Substring Match: {_avg('substring'):.2%}")
    print(f"  Token Recall: {_avg('token_recall'):.3f}")
    print(f"  ROUGE-1 F1: {_avg('rouge1_f'):.3f}")
    if 'bert_cos' in rows[0]:
        print(f"  BERT Similarity: {_avg('bert_cos'):.3f}")
    
    print(f"\n🔍 CONTEXT QUALITY:")
    if 'context_token_recall' in rows[0]:
        print(f"  Context Token Recall: {_avg('context_token_recall'):.3f}")
    if 'context_rouge1_f' in rows[0]:
        print(f"  Context ROUGE-1 F1: {_avg('context_rouge1_f'):.3f}")
    if 'context_bert_cos' in rows[0]:
        print(f"  Context BERT Similarity: {_avg('context_bert_cos'):.3f}")
    
    print(f"\n⚡ PERFORMANCE:")
    avg_lat = sum(latencies)/len(latencies)
    p95 = statistics.quantiles(latencies, n=20)[18] if len(latencies)>=20 else max(latencies)
    print(f"  Average Latency: {avg_lat*1000:.1f}ms")
    print(f"  95th Percentile Latency: {p95*1000:.1f}ms")
    print(f"  Questions per Second: {1/avg_lat:.2f}")
    
    # Save results
    if OUTPUT_CSV_PATH and rows:
        os.makedirs(OUTPUT_CSV_PATH, exist_ok=True)
        mode = "CoT" if COT_PROMPT else "Naive"
        n = len(rows)
        out_file = os.path.join(OUTPUT_CSV_PATH, f"weaviateNaive{TOP_K}_{n}.csv")
        with open(out_file, "w", encoding="utf-8", newline="") as csvf:
            writer = csv.DictWriter(csvf, fieldnames=rows[0].keys())
            writer.writeheader()
            writer.writerows(rows)
        print(f"\n✓ Results saved to: {out_file}")
else:
    print("\nNo valid results to report.")


Loaded 298 QA pairs from JSON.
🚀 Starting RAG evaluation com saída garantida em português...


Eval-best_metrics:   0%|          | 0/298 [00:00<?, ?q/s]


Q1: De acordo com o Documento Gerado #620, qual é o percurso institucional previsto para a ratificação p...
Generated: Nota: o ficheiro fornecido está identificado como Documento Gerado #733; respondo com base nesse texto.  - Percurso inst
Expected: O percurso indicado é: (1) parecer favorável do Ministério dos Negócios Estrangeiros, via Direcção-Geral dos Assuntos In
Expected File: documento_620_100.1003.docx
Retrieved Files: ['C:\\Users\\Francisco Azeredo\\Downloads\\gerador_documentos_gpt_azure (1)\\gerador_documentos_gpt_azure\\documentos_gerados\\documento_733_100.1003.docx']
Correct Doc Retrieved: False
Key Metrics: {'exact': False, 'substring': False, 'token_recall': '0.491', 'rouge1_f': '0.346', 'bert_cos': '0.810', 'context_token_recall': '0.533'}
Latency: 28767.7ms
--------------------------------------------------------------------------------

Q2: Qual é o prazo final para pagamento das Notas de Débito emitidas e o que acontecerá após esse prazo,...
Generated: Prazo: 30 di

: 

In [2]:
# Test connections before running benchmark
import requests
import time
import os

print("🔍 Testing connections...")
import weaviate
from weaviate.config import AdditionalConfig, Timeout
# Connect to Weaviate with custom timeouts

CUSTOM_INIT_TIMEOUT = 120
CUSTOM_QUERY_TIMEOUT = 900   # 15 minutes
CUSTOM_INSERT_TIMEOUT = 300  # 5 minutes
openai_api_key = os.getenv("OPENAI_API_KEY")
headers = {
    "X-Openai-Api-Key": openai_api_key
}
try:
    TIMEOUTS = Timeout(init=CUSTOM_INIT_TIMEOUT, query=CUSTOM_QUERY_TIMEOUT, insert=CUSTOM_INSERT_TIMEOUT)
    client = weaviate.connect_to_local(additional_config=AdditionalConfig(timeout=TIMEOUTS), headers=headers)
    print("✅ Weaviate client connected")
except Exception as e:
    print("Weaviate timeout config error:", e)
# Test Ollama
try:
    response = requests.get("http://localhost:11434/api/tags", timeout=5)
    if response.status_code == 200:
        models = response.json().get('models', [])
        print(f"✅ Ollama is running with {len(models)} models")
        qwen_available = any('qwen2.5' in model.get('name', '') for model in models)
        if qwen_available:
            print("✅ qwen2.5 model is available")
        else:
            print("❌ qwen2.5 model not found - run: ollama pull qwen2.5:latest")
    else:
        print(f"❌ Ollama responded with status {response.status_code}")
except Exception as e:
    print(f"❌ Ollama connection failed: {e}")
    print("Run: ollama serve")

# Test Weaviate
try:
    if 'client' in globals():
        meta = client.get_meta()
        print(f"✅ Weaviate is running, modules: {list(meta.get('modules', {}).keys())}")
        
        # Test collection exists
        if client.collections.exists("Dataset"):
            collection = client.collections.get("Dataset")
            aggregate = collection.aggregate.over_all(total_count=True)
            count = aggregate.total_count
            print(f"✅ Dataset collection has {count} documents")
            
            # Simple test query (no generation)
            try:
                test_result = collection.query.near_text(
                    query="teste", 
                    limit=1,
                    return_metadata=['distance']
                )
                print("✅ Vector search is working")
            except Exception as e:
                print(f"❌ Vector search failed: {e}")
            # Simple test generation (Portuguese enforced)
            try:
                gen_result = collection.generate.hybrid(
                    query="O que é IA?", 
                    limit=1,
                    target_vector="text_vector",
                    grouped_task="Explique brevemente em português (pt-PT): O que é IA?",
                    return_metadata=['distance']
                )
                gen_text = getattr(getattr(gen_result, 'generative', None), 'text', None)
                if gen_text:
                    if any('\u4e00' <= ch <= '\u9fff' for ch in gen_text):
                        print("⚠️ Generative search returned Chinese text; prompts will enforce pt-BR in the evaluation cell.")
                    else:
                        print("✅ Generative search is working and responding in Portuguese")
                else:
                    print("❌ Generative search returned no text")
            except Exception as e:
                print(f"❌ Generative search failed: {e}")
        else:
            print("❌ Dataset collection not found")
    else:
        print("❌ Weaviate client not initialized")
except Exception as e:
    print(f"❌ Weaviate connection failed: {e}")

print("\n🎯 If all tests pass, you can proceed with the benchmark.")
print("🎯 If tests fail, fix the issues before running the evaluation.")
client.close()

🔍 Testing connections...
✅ Weaviate client connected
✅ Weaviate client connected
✅ Ollama is running with 5 models
✅ qwen2.5 model is available
✅ Weaviate is running, modules: ['generative-anthropic', 'generative-anyscale', 'generative-aws', 'generative-cohere', 'generative-databricks', 'generative-friendliai', 'generative-google', 'generative-mistral', 'generative-nvidia', 'generative-octoai', 'generative-ollama', 'generative-openai', 'generative-xai', 'multi2multivec-jinaai', 'multi2vec-cohere', 'multi2vec-google', 'multi2vec-jinaai', 'multi2vec-nvidia', 'multi2vec-voyageai', 'reranker-cohere', 'reranker-jinaai', 'reranker-nvidia', 'reranker-voyageai', 'text2multivec-jinaai', 'text2vec-aws', 'text2vec-cohere', 'text2vec-databricks', 'text2vec-google', 'text2vec-huggingface', 'text2vec-jinaai', 'text2vec-mistral', 'text2vec-nvidia', 'text2vec-octoai', 'text2vec-openai', 'text2vec-transformers', 'text2vec-voyageai', 'text2vec-weaviate']
✅ Dataset collection has 1000 documents
✅ Vector 